# Cine21 데이터로 konlpy 테스트

In [218]:
$ jupyter notebook list

SyntaxError: invalid syntax (<ipython-input-218-a5343a1a1389>, line 1)

In [178]:
# Jupyter Notebook 화면 크기 조정 (width 값)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## 1. Import libraries

In [179]:
import sys
import time
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

from collections import Counter
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.corpus import stopwords
import gensim

import warnings
warnings.filterwarnings('ignore')

## 2. Import data

### 2.1. Import raw data

In [180]:
cine_raw_df = pd.read_csv("./cine21_final_df.csv")
cine_raw_df

,title,release_date,genre,rating,story,url,write_date,score,review
0,0.0MHz,개봉일 : 2019-05-29,공포,Unable to get rating,"에디슨의 마지막 발명품, 유령 탐지기?“증명할 수 없는 미스터리란 없다”가지 말라는...",http://www.cine21.com/movie/info/?movie_id=52988,2019-06-02,5,90년대 중반 심은하 주연 납량 특집드라마 'M' 을 다시 보는 느낌이 들었다.
1,10 미니츠 곤,개봉일 : 2019-11,"범죄, 액션",Unable to get rating,"룰이 깨진 순간, 모든 멤버가 타겟이 된다!동물적인 직감과 1초의 오차도 용납하지 ...",http://www.cine21.com/movie/info/?movie_id=55539,2019-12-28,2,전개도 너무 시시한데다 결말도 엄청 이상하게 지어서 내용이 뭐가 어떻게 돌아가는지 ...
2,100일 동안 100가지로 100퍼센트 행복찾기,개봉일 : 2019-09-12,코미디,Unable to get rating,"어렸을 적부터 함께 자라고, IT 회사도 공동 운영,사는 집도 아랫집, 윗집으로 이...",http://www.cine21.com/movie/info/?movie_id=55031,2019-09-22,6,뻔한 재미와 감동이지만 시츄에이션 설정이 독창적이었다.
3,12번째 솔저,개봉일 : 2019-04-11,전쟁,12세이상관람가,제2차 세계대전 나치에 점령된 노르웨이.나치를 함락시킬 유일한 ‘마틴 레드 작전’에...,http://www.cine21.com/movie/info/?movie_id=54607,2019-04-27,7,"제2차 세계대전 때, 나치로부터 노르웨이를 되찾기 위한 작전을 펼친 노르웨이 비밀 ..."
4,13년의 공백,개봉일 : 2019-07-04,"드라마, 가족",12세이상관람가,"“아버지가 너무 싫습니다.그런데, 조금은 좋아하는 것 같기도 합니다.”담배를 사러 ...",http://www.cine21.com/movie/info/?movie_id=54778,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1806,호라이즌 라인,개봉일 : 2021-02-10,스릴러,12세이상관람가,계기판도 GPS도 조종사도 없다!내 안의 생존 본능을 깨워라!에메랄드 빛 해안을 낀...,http://www.cine21.com/movie/info/?movie_id=57226,2021-02-22,4,재난 스릴러물은데 전혀 긴장감이 안 느껴지는게 문제
1807,호스트: 접속금지,개봉일 : 2021-07-21,공포,12세이상관람가,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01,6,맨 마지막 장면을 위한 빌드업이 조금 루즈했지만 마지막은 진짜 놀랐다.
1808,호스트: 접속금지,개봉일 : 2021-07-21,공포,12세이상관람가,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01,5,<호스트:접속금지>는 코로나 시대에 자가 격리 된 여섯 명의 친구가 화상채팅 프로그...
1809,혼자 사는 사람들,개봉일 : 2021-05-19,드라마,12세이상관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.사람들은 자꾸 말을 걸어오지만, 진아는 그저...",http://www.cine21.com/movie/info/?movie_id=57892,2021-05-23,10,시대가 좋아졌다고는 하지만 좋아진 시대를 누리는 이들은 많지 않으며 좋든 싫든 인...


In [181]:
cine_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1811 entries, 0 to 1810
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1811 non-null   object
 1   release_date  1785 non-null   object
 2   genre         1785 non-null   object
 3   rating        1726 non-null   object
 4   story         1785 non-null   object
 5   url           1785 non-null   object
 6   write_date    1152 non-null   object
 7   score         1152 non-null   object
 8   review        1152 non-null   object
dtypes: object(9)
memory usage: 127.5+ KB


In [6]:
##### rating column data compromised

### 2.2. Prepare test data
1. Pick a movie with lots of reviews
2. make a string out of all reviews

In [182]:
# len(raw_df) # 1811
# raw_df에서 리뷰 정보가 없는 영화를 뺀 work_df 만들기

# 타이틀 기준 정보 없는 영화 빼기
tmp = cine_raw_df[cine_raw_df['title'] != 'url을 찾을 수 없습니다.']
len(tmp) # 1785

1785

In [183]:
# 리뷰 정보가 NaN인 영화 633건
tmp.review.isnull().sum()

633

In [184]:
work_df = tmp.drop(tmp[tmp.review.isnull() == True].index)
len(work_df)

1152

In [185]:
# 리뷰가 한 건이라도 존재하는 영화의 수: 494편
work_df.title.nunique()

494

In [186]:
# 제목을 기준으로 리뷰 수가 가장 많은 영화 찾기: 기생충 18건
work_df.title.value_counts()

기생충            18
달콤한 인생         17
극한직업           14
함께 있을 수 있다면    12
조커             10
               ..
누구나 아는 비밀       1
해수의 아이          1
마음의 저편          1
생각의 여름          1
완벽한 가족          1
Name: title, Length: 494, dtype: int64

## 3. Konlpy code

In [159]:
stop_words= ['또','영화','분','꼭', '하', '받', '않', '말', '알','진짜', '많이', '정말', '그냥', '그', '애', '생각', '완전', '편','내용',
             '중간', '꼭','편','상미', '모두' , '거', 
 '더', '가', '뭐', '그렇', '말하', '중', '때', '되', '암', '보', '있', '개', '볼','일',
             '안','진짜', '듯', '위하', '대하', '년', '쫌', '위', '없', '주', '같', '나', '정말',
 '지', '걸', '못','좀', '그', '왜','때문', '임', '게', '것', '의', '사람', '이','대한',
             '수','이','있','하','것','들','그','되','수','보','않','나','사람','주','아니',
             '등',
             '같','우리','때','년','가','한','지','대하','오','말','일','그렇','위하','때문',
             '그것','두','말하','알','그러나','그런데','그래도','그래서','받','못하','일',
             '그런','또','문제','더','사회','많','좋','크','중','나오','가지','씨','시키',
             '만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신',
             '안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번',
             '나','다른','어떻','여자','남자','개','전','들','사실','이렇','점','싶','말','정도',
             '좀','원','잘','통하','소리','놓','영화','진짜', '많이', '정말', '그냥', '그', '애',
             '생각', '완전', '편','내용','중간', '꼭','편','상미', '모두' , '거']


In [141]:
movie_titles = work_df.title.unique().tolist()
len(movie_titles)

494

In [219]:
def get_noun(name):
    s_list = []
    s = work_df[work_df['title']==name]['review'] # input의 name과 같은 영화의 리뷰들만 추출 
    s = s.tolist() # 리뷰들을 리스트화 
    mid = []
    for i in range(0,len(s)): # 리뷰들의 형태소 분류 및 최소 길이 미만 제거 
        noun = okt.nouns(s[i])
        for i,v in enumerate(noun):
            if len(v)<2:
                noun.pop(i)
        mid+=noun

    count = Counter(mid) # 단어별 개수
    noun_list = count.most_common(100) # (리뷰 하나의)최빈 단어
    for v in noun_list: # (리뷰 하나의) 단어 list에서 불용어 제거 
        if v[0] not in stop_words:
            s_list.append(v) # 유효단어 추출후 append 
    # 리뷰별로 반복

    df= pd.DataFrame(s_list, columns =['noun', 'count'])
    df['title']=name
    return df

In [220]:
# 반복문을 돌며 영화를 검색하고 해당하는 단어의 빈도수를 상위 50개까지만 담는다.

results = []

for i in range(0,len(movie_titles)):
    results.append(get_noun(movie_titles[i]).sort_values(by='count',ascending=False).head(50))    

In [221]:
cine21_keywords = pd.concat(results).reset_index(drop=True)

KeyboardInterrupt: 

In [ ]:
cine21_keywords = cine21_keywords[['title', 'noun', 'count']]

In [ ]:
cine21_keywords

In [ ]:
# 결과 저장
cine21_keywords.to_csv("cine21_word_count.csv", index=False)

아래는 왓챠 리뷰 데이터 키워드 추출

In [168]:
watcha_df = pd.read_csv("C:/Users/Danbee/Desktop/sample-project/server/flask/utils/data/review/Wa_review_192021.csv")

In [170]:
movie_titles = watcha_df.title.unique().tolist()
len(movie_titles) # 411

results = []

for i in range(0,len(movie_titles)):
    results.append(get_noun(movie_titles[i]).sort_values(by='count',ascending=False).head(50))

In [174]:
watcha_keywords = pd.concat(results).reset_index(drop=True)
watcha_keywords = watcha_keywords[['title', 'noun', 'count']]
watcha_keywords

,title,noun,count
0,나의 특별한 형제,장애인,14
1,나의 특별한 형제,모습,9
2,나의 특별한 형제,동구,7
3,나의 특별한 형제,비장,5
4,나의 특별한 형제,애인,5
...,...,...,...
6661,올드,생략,1
6662,올드,의혹,1
6663,미드나이트,단거리,1
6664,미드나이트,육상,1


In [175]:
# 결과 저장
watcha_keywords.to_csv("watcha_word_count.csv", index=False)

### 형태소 분석기

In [195]:
greenbook_df = work_df[work_df.title == '그린 북']

In [204]:
greenbook_str = ''

for row in greenbook_df.review:
    greenbook_str += row + ' '
    
# greenbook_str

In [215]:
# 코드 출처: https://m.blog.naver.com/vangarang/221066229238

start_time = time.time()

txt = greenbook_str

okt = Okt()

print("-"*10, "형태소 분석 결과", "-"*10)

try:
    print(tagger.pos(txt))    # 품사 태깅
#     print(tagger.morphs(example)) # 형태소만 추출
#     print(okt.nouns(txt))  # 명사 추출
    
except Exception as e:
    print(e)

print("---------- %s seconds ----------" % (time.time() - start_time))

---------- 형태소 분석 결과 ----------
[('함께', 'Adverb'), ('생활', 'Noun'), ('하며', 'Verb'), ('대화', 'Noun'), ('를', 'Josa'), ('하는', 'Verb'), ('것', 'Noun'), ('만으로도', 'Josa'), ('서로', 'Noun'), ('가', 'Josa'), ('받아들여질', 'Verb'), ('수', 'Noun'), ('있다는', 'Adjective'), ('소중한', 'Adjective'), ('경험', 'Noun'), ('.', 'Punctuation'), ('다름에서', 'Adjective'), ('오는', 'Verb'), ('고통', 'Noun'), ('을', 'Josa'), ('이해', 'Noun'), ('하고자', 'Verb'), ('했던', 'Verb'), ('우정', 'Noun'), ('.', 'Punctuation'), ('more', 'Alpha'), ('두', 'Determiner'), ('남자', 'Noun'), ('의', 'Josa'), ('우정', 'Noun'), ('..', 'Punctuation'), ('감동', 'Noun'), ('..', 'Punctuation'), ('ㅎㅎ', 'KoreanParticle'), ('상당히', 'Adjective'), ('감동', 'Noun'), ('적', 'Suffix'), ('이고', 'Josa'), ('우정', 'Noun'), ('이', 'Josa'), ('넘치는', 'Adjective'), ('실화', 'Noun'), ('였습니다', 'Verb'), ('!', 'Punctuation'), ('제', 'Noun'), ('91회', 'Number'), ('오스카', 'Noun'), ('어워즈', 'Noun'), ('에서', 'Josa'), ('최우수', 'Noun'), ('작품', 'Noun'), ('상', 'Suffix'), ('을', 'Josa'), ('수상한', 'Noun'), ('영화', 'Noun

In [216]:
stemmed = okt.pos(phrase = txt, norm=True, stem=True)
not_stemmed = okt.pos(phrase = txt, norm=False, stem=False)

print(Counter(stemmed).most_common())

[(('을', 'Josa'), 46), (('이', 'Josa'), 43), (('하다', 'Verb'), 42), (('의', 'Josa'), 36), (('.', 'Punctuation'), 30), (('를', 'Josa'), 29), (('에서', 'Josa'), 23), (('에', 'Josa'), 23), (('들', 'Suffix'), 23), (('가', 'Josa'), 21), (('있다', 'Adjective'), 19), (('은', 'Josa'), 19), (('돈', 'Noun'), 17), (('토니', 'Noun'), 17), (('적', 'Suffix'), 15), (('흑인', 'Noun'), 15), (('으로', 'Josa'), 14), (('과', 'Josa'), 14), (('는', 'Josa'), 13), (('남부', 'Noun'), 12), ((',', 'Punctuation'), 11), (('아니다', 'Adjective'), 10), (('사람', 'Noun'), 10), (('되다', 'Verb'), 10), (('(', 'Punctuation'), 9), (('하고', 'Josa'), 9), ((')', 'Punctuation'), 9), (('우정', 'Noun'), 8), (('보다', 'Verb'), 8), (('같다', 'Adjective'), 8), (('인', 'Josa'), 8), (('작품', 'Noun'), 7), (('이', 'Noun'), 7), (('없다', 'Adjective'), 7), (('에게', 'Josa'), 7), (('이다', 'Josa'), 7), (('한', 'Josa'), 7), (('영화', 'Noun'), 6), (('백인', 'Noun'), 6), (('와', 'Josa'), 6), (('인종차별', 'Noun'), 6), (('편견', 'Noun'), 6), (('이탈리아', 'Noun'), 6), (('로', 'Josa'), 6), (('출신', 'Noun')

In [81]:
okt.phrases(short_example)

['영화',
 '영화 기생충',
 '어떤 평론가',
 '어떤 평론가의 말',
 '생각',
 '생각해낸 리뷰로서',
 '생각해낸 리뷰로서 기생충',
 '사회적 메세지',
 '대한',
 '대한 소소한 생각',
 '기생충',
 '깐느영화제',
 '깐느영화제 최고상',
 '수상했다는 소식',
 '국내개봉',
 '국내개봉 첫날',
 '국내개봉 첫날 열일',
 '일찍',
 '일찍 영화',
 '첫날',
 '진짜',
 '진짜 기대감',
 '진짜 기대감 만땅',
 '보고',
 '보고 난후',
 '보고 난후의 느낌',
 '깐느상',
 '거머',
 '전혀',
 '전혀 손색',
 '그 이후',
 '그 이후 여러',
 '그 이후 여러 매체',
 '연속',
 '어제',
 '어제 다시',
 '어제 다시 신랑',
 '어제 다시 신랑과 티비',
 '통해',
 '두번째',
 '긴장',
 '긴장된 상태',
 '덜 몰입',
 '두번째 볼때',
 '감정이입',
 '지하',
 '문광',
 '문광 부부',
 '문광 부부의 처참한 몰골씬',
 '슬픔',
 '눈물',
 '눈물과 키득거리는 웃음',
 '뒤엉킨채',
 '그런 영화',
 '웃음',
 '웃음 80',
 '눈물 20',
 '눈물 20 이',
 '더 큰 감동',
 '더해진 보기',
 '더해진 보기드문 수작',
 '아주',
 '아주 잘 만들어진 블랙코미디',
 '공포',
 '공포와 웃음',
 '바로',
 '바로 인',
 '바로 인 현실',
 '적나라한 묘사이기',
 '영화자체',
 '픽션',
 '그럴듯한 픽션이자',
 '리얼리스틱',
 '따름',
 '처음',
 '처음 영화',
 '때의 느낌',
 '바로 글자그대로',
 '경이로움',
 '자체',
 '채 대저택',
 '도착',
 '비디오폰',
 '대화',
 '장면',
 '정말',
 '정말 입',
 '움의 연속',
 '봉준호',
 '봉준호의 상상력',
 '바로 이',
 '바로 이 지점부터',
 '여기',
 '부터',
 '부터 지하실',
 '부터 지하실 시퀀스',
 '이 부분',
 '밋밋한 한편',
 '밋

In [18]:
# 간단한 추출기
# https://m.blog.naver.com/vangarang/221068634380

from konlpy.tag import Okt
text = parasite_review_str

def keyword_extractor(tagger, text):
    tokens = tagger.phrases(text)
    tokens = [ token for token in tokens if len(token) > 1 ] # 한 글자인 단어는 제외
    count_dict = [(token, text.count(token)) for token in tokens ]
    ranked_words = sorted(count_dict, key=lambda x:x[1], reverse=True)[:30]
    return [ keyword for keyword, freq in ranked_words ]

if __name__ == '__main__':
    tagger = Okt()
    print( keyword_extractor(twit, text) )

['영화', '지하', '기택', '사람', '그것', '기우', '부잣집', '기생', '기생충', '존재', '지하실', '로서', '식구', '사회', '작품', '네 식구', '가족', '생각', '기택네', '모습', '문제', '이기', '대한', '문광', '때문', '이들', '하나', '구조', '사장', '반 지하']


In [19]:
# TF-IDF 기반 키워드 추출기
# https://m.blog.naver.com/vangarang/221075347600

from konlpy.tag import Okt
from math import log10

def tokenizer(d):
    tagger = Okt()
    
    def keyword_extractor(text):
        tokens = tagger.phrases(text)
        tokens = [token for token in tokens if len(token) > 1]  # 한 글자인 단어는 제외
        count_dict = [(token, text.count(token)) for token in tokens]
        ranked_words = sorted(count_dict, key=lambda x: x[1], reverse=True)[:20]
        return [keyword for keyword, freq in ranked_words]
    return keyword_extractor(d)


# =======================================
# -- TF-IDF function
# =======================================

def f(t, d):
    # d is document == tokens
    return d.count(t)

def tf(t, d):
    # d is document == tokens
    return 0.5 + 0.5*f(t,d)/max([f(w,d) for w in d])

def idf(t, D):
    # D is documents == document list
    numerator = len(D)
    denominator = 1 + len([ True for d in D if t in d])
    return log10(numerator/denominator)

def tfidf(t, d, D):
    return tf(t,d)*idf(t, D)

def tokenizer(d):
    tagger = Okt()
    def keyword_extractor(text):
        tokens = tagger.phrases(text)
        tokens = [token for token in tokens if len(token) > 1]  # 한 글자인 단어는 제외
        count_dict = [(token, text.count(token)) for token in tokens]
        ranked_words = sorted(count_dict, key=lambda x: x[1], reverse=True)[:20]
        return [keyword for keyword, freq in ranked_words]
    return keyword_extractor(d)

def tfidfScorer(D):
    tokenized_D = [tokenizer(d) for d in D]
    result = []
    for d in tokenized_D:
        result.append([(t, tfidf(t, d, tokenized_D)) for t in d])
    return result

corpus = parasite_review_str

print(tokenizer(corpus))

['영화', '지하', '기택', '사람', '그것', '기우', '부잣집', '기생', '기생충', '존재', '지하실', '로서', '식구', '사회', '작품', '네 식구', '가족', '생각', '기택네', '모습']
